In [1]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 19:30:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = sc.textFile('records.txt')

In [4]:
splited_data = data.map(lambda x: frozenset(x.split()))
items = data.flatMap(lambda x: frozenset(x.split()))
kv_data = items.map(lambda x: (x, 1))
item_frequency = kv_data.reduceByKey(lambda x, y: x+y)

In [5]:
# freqeunt items
SUPPORT_S = 100
frequent_items = item_frequency.filter(lambda x: x[1]>=SUPPORT_S)
frequent_items = frequent_items.sortBy(lambda x: x[1], ascending=False)
frequent_items_list = sc.broadcast(frozenset(frequent_items.map(lambda x: x[0]).collect()))

In [6]:
# Results
print('===== 10 Most frequent items are: =====')
for i, item in enumerate(frequent_items.take(10)):
    print(f"{i+1}- {item}")
print(f'===== Number of frequent items are {frequent_items.count()} =====')

===== 10 Most frequent items are: =====
1- ('DAI62779', 6667)
2- ('FRO40251', 3881)
3- ('ELE17451', 3875)
4- ('GRO73461', 3602)
5- ('SNA80324', 3044)
6- ('ELE32164', 2851)
7- ('DAI75645', 2736)
8- ('SNA45677', 2455)
9- ('FRO31317', 2330)
10- ('DAI85309', 2293)
===== Number of frequent items are 647 =====


In [30]:
# frequent two tuples
def remove_infrequents(basket):
    return frozenset(item for item in basket if item in frequent_items_list.value)

def create_two_tuple(basket):
    items_list = list(basket)
    n = len(items_list)
    two_tuples = []
    
    for i in range(n):
        for j in range(i+1, n):
            two_tuple = tuple(sorted([items_list[i], items_list[j]]))
            two_tuples.append((two_tuple, 1))
    return two_tuples

splited_data_without_infrequents = splited_data.map(remove_infrequents)
frequent_two_tuples = splited_data_without_infrequents.flatMap(create_two_tuple).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1]>=SUPPORT_S)


In [37]:
# Results
print('===== 10 Most frequent two tuples are: =====')
for i, item in enumerate(frequent_two_tuples.sortBy(lambda x: x[1], ascending=False).take(10)):
    print(f"{i+1}- {item}")
print(f'===== Number of frequent two tuples are {frequent_two_tuples.count()} =====')

===== 10 Most frequent two tuples are: =====
1- (('DAI62779', 'ELE17451'), 1592)
2- (('FRO40251', 'SNA80324'), 1412)
3- (('DAI75645', 'FRO40251'), 1254)
4- (('FRO40251', 'GRO85051'), 1213)
5- (('DAI62779', 'GRO73461'), 1139)
6- (('DAI75645', 'SNA80324'), 1130)
7- (('DAI62779', 'FRO40251'), 1070)
8- (('DAI62779', 'SNA80324'), 923)
9- (('DAI62779', 'DAI85309'), 918)
10- (('ELE32164', 'GRO59710'), 911)
===== Number of frequent two tuples are 1334 =====


In [43]:
# frequent three tuples
def create_three_tuples(basket):
    items_list = list(basket)
    n = len(items_list)
    three_tuples = []
    
    for i in range(n):
        for j in range(i+1, n):
            for k in range(j+1, n):
                three_tuple = tuple(sorted([items_list[i], items_list[j], items_list[k]]))
                three_tuples.append((three_tuple, 1))
    return three_tuples

frequent_three_tuples = splited_data_without_infrequents.flatMap(create_three_tuples).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1]>=SUPPORT_S)

In [45]:
# Results
print('===== 10 Most frequent three tuples are: =====')
for i, item in enumerate(frequent_three_tuples.sortBy(lambda x: x[1], ascending=False).take(10)):
    print(f"{i+1}- {item}")
print(f'===== Number of frequent three tuples are {frequent_three_tuples.count()} =====')

===== 10 Most frequent three tuples are: =====


1- (('DAI75645', 'FRO40251', 'SNA80324'), 550)
2- (('DAI62779', 'FRO40251', 'SNA80324'), 476)
3- (('FRO40251', 'GRO85051', 'SNA80324'), 471)
4- (('DAI62779', 'ELE92920', 'SNA18336'), 432)
5- (('DAI62779', 'DAI75645', 'SNA80324'), 421)
6- (('DAI62779', 'ELE17451', 'SNA80324'), 417)
7- (('DAI62779', 'DAI75645', 'FRO40251'), 412)
8- (('DAI62779', 'ELE17451', 'FRO40251'), 406)
9- (('DAI75645', 'FRO40251', 'GRO85051'), 395)
10- (('DAI62779', 'FRO40251', 'GRO85051'), 381)


===== Number of frequent three tuples are 233 =====
